# IDWT (1D): $\uparrow 2 \rightarrow$ DT SYNTHESIS FILTER 
- **Keras**

[=] $IN_{rank3tensor}\rightarrow \uparrow 2 \rightarrow$ DWT DT SYNTHESIS FILTER $\rightarrow OUT_{rank3tensor}$ 

[=] 1D Reconstruction/ DECODER/ SYNTHESIS filter (input rank 3 tensor) 

[=]  Inverse Transform Layer for a rank 3 tensor

[+]  Rank 3 INPUT TENSOR $B\times N\times 1$ (bs:B, vec_dim:N, ch:1)

[=]  $g_{0}[n]$, $g_{1}[n]$ realization for $B\times N\times 1$ tensor procssing

[+]  1D CONVOLUTION: USING 1D DWT ANALYSIS FILTER impulse responses 

[+]  OTHOGONAL WAVELET FAMILES (db/sym/coif)

[=]  For IDWT of vectors (1D)

[?]  $B\times N\times ?$  -do-                 (batch_size:$B$, image_dim=$N$, channels:$multiple !!$)

<br />

--- 
Kishore Kr. Tarafdar, 
--@k 17/07/2022

COMPATIBLE WAVELETS:

<br />
<br />
<br />
<br />

---

In [1]:
import tensorflow as tf
import tensorflow.keras as keras
import keras.layers as layers
from keras.layers import Conv1D
import numpy as np
from keras.layers import Lambda
import import_ipynb

2022-09-26 04:04:10.487633: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-09-26 04:04:10.487671: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


<br />
<br />
<br />
<br />

----
## 2BPRFB Synthesis branch (IDWT)
- Upsample by 2 and Filter layer

$$ \rightarrow \uparrow 2 \rightarrow g[n] \rightarrow $$

In [2]:
import tensorflow as tf
#from keras.layers import UpSampling1D
#from keras.layers import Multiply
class SynthesisBranch1D(keras.layers.Layer):
    """1D IDWT/ Synthesis Layer ---upsample2 (zerostuffing) + FILTER --@k"""
    def __init__(self, hn:list): 
        super(SynthesisBranch1D, self).__init__()
        
        filt_len = len(hn)
        paddings = tf.constant([[0,0], [filt_len - 1, filt_len - 1],[0,0]])
        
        #+idwt filter: kernel initializer
        self.h = tf.keras.initializers.Constant(hn)
        
        #+ Padding + Conv. layers
        self.pad = Lambda(lambda x: tf.pad(x, paddings, "SYMMETRIC"))
        #self.idwt_filter = DWTConv1DLayer(1, kernel_size=filt_len, kernel_initializer=self.h, trainable=False) 
        self.idwt_filter = Conv1D(1, kernel_size=filt_len, kernel_initializer=self.h, trainable=False) #same
        

    def call(self, inputs):
        
        #+ UPSAMPLE
        x = tf.keras.layers.UpSampling1D(size=2)(inputs)
        print(x.shape[1], type(x.shape[1]),'ups+')
               
        #+ ZERO STUFFING LAYER
        c = tf.constant([[1] if i%2==0 else [0] for i in range(x.shape[1])],dtype=tf.float32)
        print(c.shape, x.shape) 
        x = Lambda(lambda x: x * c)(x)
        print(x.shape, '<')
        
        # symmetric pad (+pywt compatible) and filter 'x'
        x = self.pad(x)
        y = self.idwt_filter(x)
        
        return y#self.downsample(x)
